In [9]:
%load_ext autoreload
%autoreload 2
import torch
from cosypose.config import LOCAL_DATA_DIR
from cosypose.datasets.datasets_cfg import make_scene_dataset
from cosypose.rendering.bullet_scene_renderer import BulletSceneRenderer
from cosypose.visualization.singleview import make_singleview_prediction_plots, filter_predictions
from cosypose.visualization.singleview import filter_predictions
from bokeh.plotting import gridplot
from bokeh.io import show, output_notebook; output_notebook()
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

In [10]:
# result_id = 'tless-siso-n_views=1--318878146'
# result_id = 'tless-custom-n_views=1--4555674076' # pybullet
result_id = 'tless-custom-n_views=1--8397984787'
ds_name, urdf_ds_name = 'tless.primesense.test', 'tless.cad'
pred_key = 'pix2pose_detections/coarse/iteration=1'

results = LOCAL_DATA_DIR / 'results' / result_id / 'results.pth.tar'
scene_ds = make_scene_dataset(ds_name)
results = torch.load(results)['predictions']

0:44:40.037433 - Building index and loading annotations...


[Memory]2680.6s, 44.7min: Loading build_index...


In [11]:
results[pred_key].infos.loc[:, ['scene_id', 'view_id']].groupby('scene_id').first()

,view_id
scene_id,
1,303


In [12]:
scene_id, view_id = 1, 303

# Replace this here, you can use the dataframe above to get examples of scene/view ids.
# scene_id, view_id = 48, 733

this_preds = filter_predictions(results[pred_key], scene_id, view_id)
# this_preds.poses[0][0][-1] = -this_preds.poses[0][0][-1]
# this_preds.poses[0][1][-1] = -this_preds.poses[0][1][-1]
# this_preds.poses[1][0][-1] = -this_preds.poses[1][0][-1]
# this_preds.poses[1][1][-1] = -this_preds.poses[1][1][-1]
# this_preds.poses[2][0][-1] = -this_preds.poses[2][0][-1]
# this_preds.poses[2][1][-1] = -this_preds.poses[2][1][-1]
# this_preds.poses[3][0][-1] = -this_preds.poses[3][0][-1]
# this_preds.poses[3][1][-1] = -this_preds.poses[3][1][-1]

# opencv_to_pytorch3d = torch.tensor([[-1, 0, 0], [0, -1, 0], [0, 0, 1]], dtype=torch.float, device='cpu')
# for idx in range(len(this_preds)):
#     R = this_preds.poses[idx][:3, :3]
#     T = this_preds.poses[idx][:3, -1]

#     R = R.T
#     R = torch.matmul(R, opencv_to_pytorch3d)
#     T[0] = -T[0]
#     T[1] = -T[1]

#     # this_preds.poses[idx][:3, :3] = R
#     this_preds.poses[idx][:3, -1] = T

# obj_id = 185
# this_preds.poses = torch.unsqueeze(this_preds.poses[obj_id], 0)
# this_preds.poses_input = torch.unsqueeze(this_preds.poses_input[obj_id], 0)
# this_preds.K_crop = torch.unsqueeze(this_preds.K_crop[obj_id], 0)
# this_preds.boxes_rend = torch.unsqueeze(this_preds.boxes_rend[obj_id], 0)
# this_preds.boxes_crop = torch.unsqueeze(this_preds.boxes_crop[obj_id], 0)
# this_preds.infos = this_preds.infos[this_preds.infos.index == obj_id]

renderer = BulletSceneRenderer(urdf_ds_name)
figures = make_singleview_prediction_plots(scene_ds, renderer, this_preds)
renderer.disconnect()
# print(this_preds)

show(figures['input_im'])
show(figures['pred_overlay'])